In [ ]:
import os
import shutil
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import *
import matplotlib.pyplot as plt
import numpy as np
import json
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences

### directory to save model weights

In [ ]:
mw='./mw'

In [ ]:
os.mkdir(mw)

### loading preprocessed files

In [ ]:
with open('train_description.pkl','rb') as f:
    train_description=pickle.load(f)
with open('val_description.pkl','rb') as f:
    val_description=pickle.load(f)
with open('word_to_index.pkl','rb') as f:
    word_to_index=pickle.load(f)
with open('index_to_word.pkl','rb') as f:
    index_to_word=pickle.load(f)
with open('encoding_train.pkl','rb') as f:
    encoding_train=pickle.load(f)
with open('encoding_val.pkl','rb') as f:
    encoding_val=pickle.load(f)
embedding_idx=np.load('embedding_idx.npy',allow_pickle=True)

### calculating size of vocabulary and sentence of max_length

In [ ]:
vocab_size=len(word_to_index)+1
vocab_size

1848

In [ ]:
max_len=0
for ll in train_description.values():
    for caption in ll:
        a=caption.split()
        if(len(a)>max_len):
            max_len=len(a)
            b=a
print(max_len,b)

33 ['startseq', 'an', 'african', 'american', 'man', 'wearing', 'green', 'sweatshirt', 'and', 'blue', 'vest', 'is', 'holding', 'up', 'in', 'front', 'of', 'his', 'face', 'while', 'standing', 'on', 'busy', 'sidewalk', 'in', 'front', 'of', 'group', 'of', 'men', 'playing', 'instruments', 'endseq']


### Building model

In [ ]:
img_input=Input((2048,))
drop_img=Dropout(0.5)(img_input)
img_act=Dense(256,activation='relu')(drop_img)

In [ ]:
text_input=Input((max_len,))
emb=Embedding(vocab_size,50,mask_zero=True,weights=[embedding_idx],trainable=False)(text_input)
drog_txt=Dropout(0.5)(emb)
lstm=LSTM(256)(drog_txt)

In [ ]:
combination=Add()([img_act,lstm])
dense_1=Dense(256,activation='relu')(combination)
dense_2=Dense(vocab_size,activation='softmax')(dense_1)
model=Model(inputs=[img_input,text_input],outputs=dense_2)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 33)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 2048)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 33, 50)       92400       input_2[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 2048)         0           input_1[0][0]                    
______________________________________________________________________________________________

In [ ]:
#train_description-> img_id:sents
#word_to_index-> word:index
#encoding_train-> img_id:(2048,)

In [ ]:
def data_generator(train_description,word_to_index,encoding_train,batch_size=3,max_len=35):
    x1,x2,y=[],[],[]
    n=0
    while True:
        for key,desc_list in train_description.items():
            n+=1
            photo=encoding_train[key]
            for desc in desc_list:
                desc=[word_to_index[words] for words in desc.split() if words in word_to_index]
                for i in range(1,len(desc)):
                    xi=desc[:i]
                    yi=desc[i]
                    xi=pad_sequences([xi],maxlen=max_len,padding='post',truncating='post')[0]
                    yi=to_categorical([yi],num_classes=vocab_size)[0]
                    x1.append(photo)
                    x2.append(xi)
                    y.append(yi)
            if(n==batch_size):
                x1=np.array(x1)
                x2=np.array(x2)
                y=np.array(y)
                yield([x1,x2],y)
                x1,x2,y=[],[],[]
                n=0

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [ ]:
epochs=30
batch_size=3
steps=len(train_description)//batch_size
val_steps=len(val_description)//batch_size

### training for first 30 epochs at fast learning rate

In [ ]:
for i in range(epochs):
    gen=data_generator(train_description,word_to_index,encoding_train,batch_size,max_len)
    val_gen=data_generator(val_description,word_to_index,encoding_val,batch_size,max_len)
    print('epochs {}/{}'.format(i+1,epochs))
    model.fit_generator(gen,epochs=1,steps_per_epoch=steps,validation_data=val_gen, validation_steps=val_steps)
    model.save('./mw/epochs'+str(i+1)+'.h5')

epochs 1/30
2000/2000 [==============================] - 27s 14ms/step - loss: 3.9663 - val_loss: 3.5815
epochs 2/30
2000/2000 [==============================] - 26s 13ms/step - loss: 3.4556 - val_loss: 3.4915
epochs 3/30
2000/2000 [==============================] - 26s 13ms/step - loss: 3.3045 - val_loss: 3.4576
epochs 4/30
2000/2000 [==============================] - 26s 13ms/step - loss: 3.2046 - val_loss: 3.4571
epochs 5/30
2000/2000 [==============================] - 27s 13ms/step - loss: 3.1323 - val_loss: 3.4752
epochs 6/30
2000/2000 [==============================] - 27s 13ms/step - loss: 3.0745 - val_loss: 3.4941
epochs 7/30
2000/2000 [==============================] - 27s 13ms/step - loss: 3.0247 - val_loss: 3.5028
epochs 8/30
2000/2000 [==============================] - 27s 13ms/step - loss: 2.9848 - val_loss: 3.5437
epochs 9/30
2000/2000 [==============================] - 27s 13ms/step - loss: 2.9518 - val_loss: 3.5696
epochs 10/30
2000/2000 [==============================]

### training for rest 20 epochs as lower learning rate but larger batch size

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(0.0001))
model.load_weights('model_weights/epochs30.h5')

In [ ]:
batch_size=6
steps=len(train_description)//batch_size
val_steps=len(val_description)//batch_size

In [ ]:
for i in range(30,50):
    gen=data_generator(train_description,word_to_index,encoding_train,batch_size,max_len)
    val_gen=data_generator(val_description,word_to_index,encoding_val,batch_size,max_len)
    print('epochs {}/{}'.format(i+1,50))
    model.fit_generator(gen,epochs=1,steps_per_epoch=steps,validation_data=val_gen, validation_steps=val_steps)
    model.save('./mw/epochs'+str(i+1)+'.h5')

epochs 31/50
1000/1000 [==============================] - 22s 22ms/step - loss: 2.6700 - val_loss: 3.9165
epochs 32/50
1000/1000 [==============================] - 21s 21ms/step - loss: 2.6109 - val_loss: 3.9159
epochs 33/50
1000/1000 [==============================] - 21s 21ms/step - loss: 2.5894 - val_loss: 3.9159
epochs 34/50
1000/1000 [==============================] - 21s 21ms/step - loss: 2.5770 - val_loss: 3.9198
epochs 35/50
1000/1000 [==============================] - 21s 21ms/step - loss: 2.5680 - val_loss: 3.9238
epochs 36/50
1000/1000 [==============================] - 21s 21ms/step - loss: 2.5590 - val_loss: 3.9286
epochs 37/50
1000/1000 [==============================] - 21s 21ms/step - loss: 2.5513 - val_loss: 3.9299
epochs 38/50
1000/1000 [==============================] - 21s 21ms/step - loss: 2.5438 - val_loss: 3.9351
epochs 39/50
1000/1000 [==============================] - 21s 21ms/step - loss: 2.5373 - val_loss: 3.9361
epochs 40/50
1000/1000 [======================

### Caption generation

In [ ]:
base_model = ResNet50(weights='imagenet')
rn = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

102973440/102967424 [==============================] - 2s 0us/step


In [ ]:
def preprocess(img):
    x = np.expand_dims(img, axis=0)
    x = preprocess_input(x)
    preds = rn.predict(x)
    return preds.flatten()

In [ ]:
def caption(photo):
    start = 'startseq'
    for i in range(max_len):
        sequence = [word_to_index[w] for w in start.split() if w in word_to_index]
        sequence = pad_sequences([sequence], maxlen=max_len)
        y_pred = model.predict([photo,sequence], verbose=0)
        y_pred = np.argmax(y_pred)
        word = index_to_word[y_pred]
        start += ' ' + word
        if word == 'endseq':
            break
    sents = start.split()
    sents = sents[1:-1]
    sents = ' '.join(sents)
    return sents

In [ ]:
def gen_caption(path):
    img = image.load_img(path,target_size=(224,224))
    img= image.img_to_array(img,dtype='uint8')
    photo=preprocess(img)
    plt.imshow(img)
    plt.show()
    print(caption(photo.reshape((1,-1))))